In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import pymc as pm

In [ ]:
df_horse_race_results = pd.read_csv(r'd:\Data\BFTrader\horseresults.csv', sep='|', dtype={'Jockey': object, 'Trainer': object, 'Sex': 'category', 'status': 'category'} )
#df_horse_race_results.to_parquet(r'd:\Data\BFTrader\horseresults.parquet')

In [ ]:
df_runners = df_horse_race_results[df_horse_race_results['status']=='RUNNER'].copy()
df_runners['finishPosition'] = df_runners['finishPosition'].map(lambda x: np.nan if x == 0 else x)
df_runners = df_runners.sort_values(['raceId', 'finishPosition'])
df_runners.head(10)

In [ ]:
df_runners_known_position = df_runners[~np.isnan(df_runners['finishPosition'])]
df_runners_known_position.head(10)

In [ ]:
df_left = df_runners_known_position[['raceId','runnerId']].shift()
df_right = df_runners_known_position[['raceId','runnerId']]
df_idx = df_left.join(df_right, lsuffix='_l', rsuffix='_r').dropna()
df_idx['raceId_l'] = df_idx['raceId_l'].map(int)
df_idx['runnerId_l'] = df_idx['runnerId_l'].map(int)
df_idx = df_idx[df_idx['raceId_l']==df_idx['raceId_r']]
display(df_idx.head(15))
higher_idx = df_idx['runnerId_l'].values
lower_idx = df_idx['runnerId_r'].values
observations = [1 for i in higher_idx]
runners_count = max([max(higher_idx),max(lower_idx)]) + 1

In [ ]:
horse_ids = ['a','b','c','d']
trainer_ids = ['x','y','z']
results = [['a','b','c'],['b','d','a','c']]
results_idx = [[0,1,2],[1,3,0,2]]

In [ ]:
with pm.Model() as model:
    horse_skills = pm.Normal('horse_skills', shape=(runners_count,))
    #trainer_skills = pm.Normal('trainer_skills', shape=len(trainer_ids))
    p = pm.math.sigmoid(horse_skills[higher_idx] - horse_skills[lower_idx])
    observed = pm.Bernoulli('observed', p = p, observed=observations)
model

In [ ]:
with model:
    trace = pm.sample(1000, tune=100, cores=1)

In [ ]:
#pm.traceplot(trace)